# Unsupervised Machine Scoring of Free Response Answers—Validated Against Law School Final Exam Questions

This notebook implements the methods described in _Unsupervised Machine Scoring of Free Response Answers—Validated Against Law School Final Exam Questions_, to be presented at the [Computational Legal Studies Conference](https://cclaw.smu.edu.sg/events/computational-legal-studies-2022), March 2022, hosted by the Center for Computational Law at Singapore Management University.

Paper Summary

> This paper presents a novel method for unsupervised machine scoring of short answer and essay question responses, relying solely on a sufficiently large set of responses to a common prompt, absent the need for pre-labeled sample answers—given said prompt is of a particular character. That is, for questions where “good” answers look similar, “wrong” answers are likely to be “wrong” in different ways. Consequently, when a collection of text embeddings for responses to a common prompt are placed in an appropriate feature space, the centroid of their placements can stand in for a model answer, providing a lodestar against which to measure individual responses. This paper examines the efficacy of this method and discusses potential applications.
>
>Current methods for the automated scoring of short answer and essay questions are poorly suited to spontaneous and idiosyncratic assessments. That is, the time saved in grading must be balanced against the time required for the training of a model. This includes tasks such as the creation of pre-labeled sample answers. This limits the utility of machine grading for single classes working with novel assessments. The method described here eliminates the need for the preparation of pre-labeled sample answers. It is the author’s hope that such a method may be leveraged to reduce the time needed to grade free response questions, promoting the increased adoption of formative assessment esp. in contexts like law school instruction which traditionally have relied almost exclusively on summative assessments.
>
>Ranking by the algorithm is found to be statistically significant when compared to a pseudo-random shuffle. To determine how similar a list’s order was to that produced by a human grader, the lowest number of neighbor swaps needed to transform the ordering of these lists into that of the human ordering was calculated. For a dataset including more than one thousand student answers to a set of thirteen free response questions, drawn from six Suffolk University Law School final exams, taught by five instructors, the p-value for a paired t-test of the two populations’ swaps, with the pseudo-random group acting as the untreated group and the machine-grader acting as the treatment, came to 0.000000991, allowing us to reject the null hypothesis that the machine's ordering is equivalent to a random shuffle. Additionally, the Cohen’s d for the number of swaps between the pseudo-random ordering and machine ordering was found to be large (i.e., 0.988).

This notebook was used to obtain the results referenced above. Student exam answers and their associated grade data were acquired and this research conducted after approval was granted by Suffolk University’s Office of Research and Sponsored Programs (ORSP) which oversees all human subject research at Suffolk University.

This notebook implements the primary steps of the method described in the paper, namely:  

1. Produce an embedding for each answer that captures as much of the relevant information as possible.
2. Find the centroid for all of the embeddings in your population of answers and impute the location of a  “correct” model answer.
3. Measure the distance between each answer’s embedding and the “correct” answer (e.g., the answers’ centroid or medoid).
4. Convert the answers’ distances from the model answer into z-scores for the population of answers.
5. Translate these z-scores into some known grading scale.
6. Order the answers according to this scale.
7. Compare these orderings to random orderings by seeing how many times you have to change their rankings to obtain the same ordering as one would get if they were ordered by their human-assigned grades.

_Note: The methods described here are the subject of USPTO Patent Application Serial No. 17/246,563._

## Contents

_These, and other internal links (e.g., "back to contents"), may not work if you are viewing the preview of this notebook on GitHub.com. Links to other notebooks, external sites, and exam questions should, however, be okay._

This notebook has three sections:

- [Data: Student Exam Answers](#Data:-Student-Exam-Answers)
- [Code:Code: Libraries and Novel Functions](#Code:-Libraries-and-Novel-Functions)
- [Results](#Results)


## Data: Student Exam Answers

More than one thousand exam answers were obtained as PDF files containing answers to thirteen free response questions, drawn from six Suffolk University Law School final exams, taught by five instructors. Each PDF corresponded to a single student exam. These PDFs were parsed to extract their answers, their contents converted into XML of the following format.

```
<EXAM>
    <STUDENT id='ID'>00000000</STUDENT>
    <QUESTION id='Q1'>
        <![CDATA[
            text of written answer to question one
        ]]>
    </QUESTION>
    <QUESTION id='Q2'>
        <![CDATA[
            text of written answer to question two
        ]]>
    </QUESTION>
    <QUESTION id='Q3'>
        <![CDATA[
            text of written answer to question three
        ]]>
    </QUESTION>
</EXAM>
```

These translated files were reviewed by hand and reformatted as needed to correct any formatting errors. Each XML file was then read into a csv file for its associated exam as a single row with their columns corresponding to each of its elements (e.g., ID, Q1, Q2). Additionally, a column stating the number of words contained in each question was appended to the csv file (e.g., size_Q1, size_Q2). E.g., 

|ID|Q1|Q2|size_Q1|size_Q2|
|--|--|--|-------|-------|
|00001|text of 1's ans to q1|text of 1's ans to q2|6|6|
|00002|text of 2's ans to q1|text of 2's ans to q2|6|6|

Instructors also provided scores for each exam question. These were placed in a csv for each exam with the scores on each question associated to the exam ID. E.g., 

|ID|Q1|Q2|
|--|--|--|
|00001|96|93|
|00002|83|89|

To see the code used to assist in this processing see the following notebook: [Prep Exams](Prep%20Exams.ipynb).

In keeping with the wishes of those instructors who provided exams, only three of the exams are available here for review (i.e., [property_instructor_A](https://colarusso.github.io/free-response-scoring/exam_questions/property_instructor_A.docx), [property_instructor_B](https://colarusso.github.io/free-response-scoring/exam_questions/property_instructor_B.docx), and [crim_instructor_E](https://colarusso.github.io/free-response-scoring/exam_questions/crim_instructor_E.docx)). Another two are on file with the author and may be shared upon request and the assent of their authors. In keeping with the author's wishes, the remaining exam will not be shared. Subject to constraints imposed by the Family Educational Rights and Privacy Act (FERPA), three of the answer sets may be shared upon request. One of these is linked to an exam requiering instructor assent to be shared.

Please note that the exam questiones shared at the links above are docx files and have been redacted to exclude the instructor's name and text that does not include the scored question prompts (e.g., multiple choice questions). 

To facilitate retrieval of the exam answers which were stored in various folders, a list of dictionaries is defined. Each dictionary defines the folder name where csv files can be found as well as the names of its relevant columns for later consideration (i.e., their ID and those questions to be scored). To avoid sharing of this data publicly, as seen below, these files are not included in this repository and were located outside of this repository's folder (i.e., `../data/`). 

In [1]:
exams = [
            {"folder":"../data/property_instructor_A","columns":["ID","SHORT_ANS","Q1","Q2"]},
            {"folder":"../data/property_instructor_B","columns":["ID","Q1","Q2"]},
            {"folder":"../data/environ_instructor_B","columns":["ID","Q2"]},
            {"folder":"../data/PR_instructor_C","columns":["ID","Q1","Q2"]},
            {"folder":"../data/contracts_instructor_D","columns":["ID","Q1","Q2","Q3"]},
            {"folder":"../data/crim_instructor_E","columns":["ID","Q1","Q2"]}
          ]

[back to contents](#Contents)

## Code: Libraries and Novel Functions

### Python Libraries et al.

First we'll load the following libraries. These are needed to perform our scoring. 

In [2]:
import csv
import pandas as pd
import numpy as np
from numpy import var
from math import sqrt
import spacy
from scipy import stats
from sklearn.preprocessing import normalize

# https://spacy.io/models/en#en_core_web_lg
nlp1 = spacy.load('en_core_web_lg') 
# https://github.com/explosion/spacy-transformers/tree/88814f5f4be7f0d4c784d8500c558d9ba06b9a56
nlp2 = spacy.load("en_trf_distilbertbaseuncased_lg") 
nlp3 = spacy.load("en_trf_robertabase_lg") 

In [3]:
def score_exams(exams,model='nlp1',normv=0,score=0,goal="centroid",runs=1):

    # list of swaps needed to move from machine-scored ordering to that of the graded exams
    M = []  
    # list of swaps needed to move from machine-scored ordering to that of the graded exams
    R = []

    # for every exam in the list of exams  
    for exam in exams:

        print("\n=========================================================")
        print(exam["folder"])
        print("=========================================================\n")

        # load this exam's answer texts
        texts = pd.read_csv("%s/texts.csv"%exam["folder"], encoding="utf-8")
        # load this exam's grades
        actual = pd.read_csv("%s/actual.csv"%exam["folder"], encoding="utf-8")

        # print the relevant columns for this exam (i.e., ID and questions to score)
        print(exam["columns"])

        # score the questions listed above
        output_df = grade(texts[exam["columns"]],model,normv,score,goal)

        # only include grades for those questions defined above
        actual = actual[[x for x in actual.columns if x in exam["columns"]]]

        # merge the machine-scored dataframe with that of the actual grades
        df_exam = actual.merge(output_df, on="ID")
        
        # display a preview of the two scores (first few rows of the merged dataframe)
        # I'm excluding the ID here to avoid sharing IDs in the cell's output
        display(df_exam[[x for x in list(df_exam.columns.values) if (x not in ["ID"])]].head())

        print("Number of entries:",len(df_exam))

        # for each of the questions that were scored
        for qn in [x for x in exam["columns"] if (x not in ["ID"])]:
            mean_words = int(round(texts["size_%s"%qn].mean()))
            print("\n%s\nMean Words:"%qn,mean_words)

            # create a dataframe with the grade and machine scores for this question
            df = df_exam[["ID", "%s_x"%qn, "%s_y"%qn]]

            # ideally the granularity of the machine and human scoring should be the same
            # to avoid false precision. It may be worth imposing this in some manner 
            # simmilar to this comment. 
            # df["%s_y"%qn] = round(df["%s_x"%qn].max()*(df["%s_y"%qn]/100))

            # sort answers by the machine scores
            df = df.sort_values(by='%s_y'%qn, ascending=True)

            # create an list of these scores then count the swaps needed to bring them into 
            # agreement with the grades. Append this to our list of swaps needed to move from 
            # machine-scored ordering to that of the graded exams
            arr_ = df[df.columns[1]].values.copy()
            n = len(arr_)
            M_o = countSwaps(arr_, n)
            M.append(M_o)

            # we're going to do the same as the above but for pseudo-random shuffles.
            # we'll shuffle and count swaps for the number of times defined in `runs`
            # and average the results 
            tmp = []
            for x in range(runs):
                # note the use of `sample.` This is where we get our pseudo-random ordering
                arr_ = df[df.columns[1]].sample(frac=1).values.copy()
                n = len(arr_)
                R_o = countSwaps(arr_, n)
                tmp.append(R_o)

            R_o = np.array(tmp).mean()
            R.append(R_o)

            print("Algo score, swaps needed:",M_o)
            print("Swaps needed for first 10 out of %s pseudo-random runs:"%runs)
            print(tmp[:10])
            print("Pseudo-random, avergae swaps needed:",R_o)

    print("\n=========================================================")
    print("Swaps (Machine):")
    print(M)
    print("N:",len(M),"\tMean: ",np.mean(M),"\tVar: ",np.var(M))
    print("Avergae swaps (Pseudo-random):")
    print(R)
    print("N:",len(R),"\tMean: ",np.mean(R),"\tVar: ",np.var(R),"\n")

    # Print out a summary table of the percent difference between pseudo-random and machine swaps
    pdiff = []
    i = 0
    for x in R:
        if x != 0 and M[i] != 0:
            pdiff_ = 100*(x-M[i])/((x+M[i])/2)
            pdiff.append(pdiff_)
            print( i+1,")",x,">",M[i],"=>",pdiff_ )
        else:
            print("error")
        i = i + 1

    print("\nAvergae percent difference between swaps needed for pseudo-random and machine sorting:")
    print(np.array(pdiff).mean(),"\n")
        
    # run t test for the two populations of swaps
    sample_n = len(M)
    print(stats.ttest_rel(M[:sample_n], R[:sample_n]))
    # return the effect size
    print("Cohen's d: ",cohend(R[:sample_n], M[:sample_n]))

In [4]:
def grade(df,model,normv,score=0,goal="centroid"):
       
    df = df.copy()    
    
    # exclude the ID column to get only the answer texts
    questions = [x for x in list(df.columns.values) if (x not in ["ID"])]
    # vectorize the text of each answer
    for x in questions:
        df["%s_vec"%x] = df[x].apply(vectorize, args=(model,normv)) 
    
    # for each question
    for x in questions:
        
        # based on the `goal` parameter grade, against either the centroid or medoid
        # that is, set either as the model answer aginst which to grade others
        if goal=="centroid":
            # centroid 
            best_ans = df["%s_vec"%x].mean()
        else:
            # medoid 
            best_ans = df["%s_vec"%x].iloc[best_answer(df,x)]

        # measure each answer's distance from the model answer above
        df["%s_dist"%x] = df["%s_vec"%x].apply(distance, args=(best_ans,))
        
        # replace the text of the answer with its z-score
        df[x] =  df["%s_vec"%x].apply(z_score, args=(df,x,best_ans))
        
        # translate the z-score into a standard grade using
        # the translation defined by `score`
        if score==0:
            df[x] =  df[x].apply(score_0)
        elif score==1:
            df[x] =  df[x].apply(score_1)
            
    # create output df without the vectors or distance measures
    output_df = df[[ x for x in list(df.columns.values) if ("_vec" not in x) and ("_dist" not in x) ]].copy()
            
    return output_df

In [5]:
def vectorize(row,model='nlp1',normv=0): 

    # use a method for vectorization based on the `model` parameter
    # note: some methods can work across the entire text of a qustion
    # others requier vectorization at the sentence level
    if model=='nlp1':
        nlp = nlp1
        mode = 0
    elif model=='nlp2':
        nlp = nlp2
        mode = 1
    elif model=='nlp3':
        nlp = nlp3
        mode = 1
        
    try:
        if mode == 1:
            # vectorize at the sentence level
            doc = nlp(row)
            tmp_df = pd.DataFrame({'sent' : []})
            i=0 
            for sent in doc.sents:
                sent_vec = nlp(sent.string.strip()).vector
                # if normv=1 normalize each sentence vector
                if normv==1:
                    sent_vec = norm(sent_vec)
                if i==0:
                    tmp_df = pd.DataFrame([[sent_vec]],columns=["sent"])
                else:
                    tmp_df = tmp_df.append(pd.DataFrame([[sent_vec]],columns=["sent"]), ignore_index=True)
                i+=1
            # average sentence vectors 
            output = tmp_df["sent"].mean()
        else:
            # vectorize all input text together
            output = nlp(row).vector

        # if normv=1 normalize the answer's vector
        if normv==1:
            output = norm(output)  
    
        return output
    except:
        return np.NaN
    
def norm(row):
    # normalize vector
    matrix = row.reshape(1,-1).astype(np.float64)
    return normalize(matrix, axis=1, norm='l1')[0]

In [6]:
def best_answer(df,column):
    # Return the medoid (like the centroid, but confined to a member of the group) 
    # i.e., the best answer in the group of answers.
    tmp_df = df[[ "%s_vec"%column ]].copy()
    centroid = tmp_df["%s_vec"%column].mean()
    tmp_df["%s_dist"%column] = tmp_df["%s_vec"%column].apply(distance, args=(centroid,))
    tmp_df = tmp_df[tmp_df["%s_dist"%column]==tmp_df["%s_dist"%column].min()]
    return tmp_df.index[0]

def distance(row,centroid):
    # calculate the Euclidean distance between centroid and row
    # See https://stackoverflow.com/a/1401828
    return np.linalg.norm(centroid-row)

In [7]:
def z_score(row,df,x,best):
    # calculate the z-score for a given distance
    return -1*(np.linalg.norm(best-row)-df["%s_dist"%x].mean())/df["%s_dist"%x].std()

In [8]:
# Define grade bin boarders based on z-score
# --------------------------------------
# There's no standard mapping between z-scores and letter grades
# However, it's common for schools to set their own curve which 
# is in the same spirit. Therefor I have created a mapping where
# the average answer recieves a B. Others might choose differently

grade_bins = [
              2,    # A+
              1.5,  # A
              1,    # A-
              0.5,  # B+
              0,    # B
              -0.5, # B-
              -1,   # C+
              -1.5, # C
              -2,   # C-
              -2.5, # D+
              -3,   # D
              -3.5  # D-
             ]

# Assign numeric grades based on z-score
# -----------------------------------------
# This was the original z-score to percentage grade translation.
# I took the above and translated it into numberic grades
# with the scale bottoming out at 59 and topping out at 97
def score_0(row):
    if row >= grade_bins[0]:
        grade = 97
    elif row >= grade_bins[1]:
        grade = 93+((row-grade_bins[1])/(grade_bins[0]-grade_bins[1]))*4
    elif row >= grade_bins[2]:
        grade = 90+((row-grade_bins[2])/(grade_bins[1]-grade_bins[2]))*3
    elif row >= grade_bins[3]:
        grade = 87+((row-grade_bins[3])/(grade_bins[2]-grade_bins[3]))*3
    elif row >= grade_bins[4]:
        grade = 83+((row-grade_bins[4])/(grade_bins[3]-grade_bins[4]))*4
    elif row >= grade_bins[5]:
        grade = 80+((row-grade_bins[5])/(grade_bins[4]-grade_bins[5]))*3
    elif row >= grade_bins[6]:
        grade = 77+((row-grade_bins[6])/(grade_bins[5]-grade_bins[6]))*3
    elif row >= grade_bins[7]:
        grade = 73+((row-grade_bins[7])/(grade_bins[6]-grade_bins[7]))*4
    elif row >= grade_bins[8]:
        grade = 70+((row-grade_bins[8])/(grade_bins[7]-grade_bins[8]))*3
    elif row >= grade_bins[9]:
        grade = 67+((row-grade_bins[9])/(grade_bins[8]-grade_bins[9]))*3
    elif row >= grade_bins[10]:
        grade = 63+((row-grade_bins[10])/(grade_bins[9]-grade_bins[10]))*4
    elif row >= grade_bins[11]:
        grade = 60+(row-grade_bins[11])*(3/(grade_bins[10]-grade_bins[11]))
    else:
        grade = 59
        
    # The final output is rounded to avoid false percision. A better 
    # approach is to bin all of the scores. This is done in score_1 below
    return round(grade)

# Assign numeric grades based on z-score
# -----------------------------------------
# Upon subsequent reflection, I realized that binned scores were preferable
# to a continuous score with dropoffs at 59 and 97. The above translation,
# however, was used on the first run of this code and so is included here
# for compleatness.  
def score_1(row):
    if row >= grade_bins[0]:
        grade = 97
    elif row >= grade_bins[1]:
        grade = 93
    elif row >= grade_bins[2]:
        grade = 90
    elif row >= grade_bins[3]:
        grade = 87
    elif row >= grade_bins[4]:
        grade = 83
    elif row >= grade_bins[5]:
        grade = 80
    elif row >= grade_bins[6]:
        grade = 77
    elif row >= grade_bins[7]:
        grade = 73
    elif row >= grade_bins[8]:
        grade = 70
    elif row >= grade_bins[9]:
        grade = 67
    elif row >= grade_bins[10]:
        grade = 63
    elif row >= grade_bins[11]:
        grade = 60
    else:
        grade = 59
    return grade

In [9]:
# The following functions are used to count the number of adjacent swaps needed
# to change one ordering into another. It was written by Shivam Gupta
# See https://www.geeksforgeeks.org/number-swaps-sort-adjacent-swapping-allowed/
# -----------------------------------------------------
# python 3 program to count number of swaps required 
# to sort an array when only swapping of adjacent 
# elements is allowed. 
# include <bits/stdc++.h> 

#This function merges two sorted arrays and returns inversion count in the arrays.*/ 
def merge(arr, temp, left, mid, right): 
    inv_count = 0

    i = left #i is index for left subarray*/ 
    j = mid #i is index for right subarray*/ 
    k = left #i is index for resultant merged subarray*/ 
    while ((i <= mid - 1) and (j <= right)): 
        if (arr[i] <= arr[j]): 
            temp[k] = arr[i] 
            k += 1
            i += 1
        else: 
            temp[k] = arr[j] 
            k += 1
            j += 1

            #this is tricky -- see above explanation/ 
            # diagram for merge()*/ 
            inv_count = inv_count + (mid - i) 

    #Copy the remaining elements of left subarray 
    # (if there are any) to temp*/ 
    while (i <= mid - 1): 
        temp[k] = arr[i] 
        k += 1
        i += 1

    #Copy the remaining elements of right subarray 
    # (if there are any) to temp*/ 
    while (j <= right): 
        temp[k] = arr[j] 
        k += 1
        j += 1

    # Copy back the merged elements to original array*/ 
    for i in range(left,right+1,1): 
        arr[i] = temp[i] 

    return inv_count 

#An auxiliary recursive function that sorts the input 
# array and returns the number of inversions in the 
# array. */ 
def _mergeSort(arr, temp, left, right): 
    inv_count = 0
    if (right > left): 
        # Divide the array into two parts and call 
        #_mergeSortAndCountInv() 
        # for each of the parts */ 
        mid = int((right + left)/2) 

        #Inversion count will be sum of inversions in 
        # left-part, right-part and number of inversions 
        # in merging */ 
        inv_count = _mergeSort(arr, temp, left, mid) 
        inv_count += _mergeSort(arr, temp, mid+1, right) 

        # Merge the two parts*/ 
        inv_count += merge(arr, temp, left, mid+1, right) 

    return inv_count 

#This function sorts the input array and returns the 
#number of inversions in the array */ 
def countSwaps(arr, n): 
    temp = [0 for i in range(n)] 
    return _mergeSort(arr, temp, 0, n - 1) 

In [10]:
# The following function calculates Cohen d (the effect size)
# based on two populations. It was written by Jason Brownlee
# See https://machinelearningmastery.com/effect-size-measures-in-python/
# function to calculate Cohen's d for independent samples
def cohend(d1, d2):
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
    # calculate the pooled standard deviation
    s = np.sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    # calculate the means of the samples
    u1, u2 = np.mean(d1), np.mean(d2)
    # calculate the effect size
    return (u1 - u2) / s

[back to contents](#Contents)

# Results

The arrangement of parameters below corresponds to those used when all of the exam data was first run through the above scoring method. To avoid biasing parameter selection, prior work on the method did NOT use the exam data referenced. You will note that the count of swapps needed for the pseudo-random shuffle differs from that in Table 1. That is because the output here represents a subsequent run of the method. To obtain the values found in table 2, the pseudo-random shuffle was repeated 100,000 times, and the number of swaps were averaged. These results can be found in the cell after the one below. 

In [11]:
score_exams(exams,model='nlp1',normv=0,score=0,goal="centroid",runs=1)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,86,83,87
1,10,30,32,88,86,87
2,12,39,24,90,90,85
3,15,38,16,88,87,86
4,13,43,19,91,89,87


Number of entries: 81

SHORT_ANS
Mean Words: 432
Algo score, swaps needed: 996
Swaps needed for first 10 out of 1 pseudo-random runs:
[1644]
Pseudo-random, avergae swaps needed: 1644.0

Q1
Mean Words: 2051
Algo score, swaps needed: 1017
Swaps needed for first 10 out of 1 pseudo-random runs:
[1367]
Pseudo-random, avergae swaps needed: 1367.0

Q2
Mean Words: 952
Algo score, swaps needed: 828
Swaps needed for first 10 out of 1 pseudo-random runs:
[1492]
Pseudo-random, avergae swaps needed: 1492.0

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,88,86
1,20.0,18.0,80,84
2,24.0,25.0,89,80
3,18.0,22.0,83,91
4,17.0,17.0,78,79


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1290
Swaps needed for first 10 out of 1 pseudo-random runs:
[1776]
Pseudo-random, avergae swaps needed: 1776.0

Q2
Mean Words: 1546
Algo score, swaps needed: 1326
Swaps needed for first 10 out of 1 pseudo-random runs:
[1566]
Pseudo-random, avergae swaps needed: 1566.0

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 911
Swaps needed for first 10 out of 1 pseudo-random runs:
[1275]
Pseudo-random, avergae swaps needed: 1275.0

Q2
Mean Words: 570
Algo score, swaps needed: 795
Swaps needed for first 10 out of 1 pseudo-random runs:
[1277]
Pseudo-random, avergae swaps needed: 1277.0

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,87,87,81
1,53,27,27,82,80,89
2,38,36,20,81,84,81
3,66,37,37,83,88,87
4,71,34,38,85,88,85


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1178
Swaps needed for first 10 out of 1 pseudo-random runs:
[1388]
Pseudo-random, avergae swaps needed: 1388.0

Q2
Mean Words: 1050
Algo score, swaps needed: 993
Swaps needed for first 10 out of 1 pseudo-random runs:
[1340]
Pseudo-random, avergae swaps needed: 1340.0

Q3
Mean Words: 839
Algo score, swaps needed: 807
Swaps needed for first 10 out of 1 pseudo-random runs:
[1483]
Pseudo-random, avergae swaps needed: 1483.0

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,89,89
1,24.0,25.0,82,85
2,26.5,24.0,63,75
3,23.5,23.0,87,87
4,26.5,21.0,87,90


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1792
Swaps needed for first 10 out of 1 pseudo-random runs:
[1978]
Pseudo-random, avergae swaps needed: 1978.0

Q2
Mean Words: 2137
Algo score, swaps needed: 1637
Swaps needed for first 10 out of 1 pseudo-random runs:
[1957]
Pseudo-random, avergae swaps needed: 1957.0

Swaps (Machine):
[996, 1017, 828, 1290, 1326, 115, 911, 795, 1178, 993, 807, 1792, 1637]
N: 13 	Mean:  1052.6923076923076 	Var:  163027.44378698224
Avergae swaps (Pseudo-random):
[1644.0, 1367.0, 1492.0, 1776.0, 1566.0, 159.0, 1275.0, 1277.0, 1388.0, 1340.0, 1483.0, 1978.0, 1957.0]
N: 13 	Mean:  1438.6153846153845 	Var:  188033.62130177513 

1 ) 1644.0 > 996 => 49.09090909090909
2 ) 1367.0 > 1017 => 29.36241610738255
3 ) 1492.0 > 828 => 57.241379310344826
4 ) 1776.0 > 1290 => 31.702544031311156
5 ) 1566.0 > 1326 => 16.597510373443985
6 ) 159.0 > 115 => 32.11678832116788
7 ) 1275.0 > 911 => 33.3028362305581
8 ) 1277.0 > 795 => 46.52509652509652
9 ) 1388.

As stated above, the following cell is the same as the above except the pseudo-random shuffle was run 100,000 times and the swaps needed averaged. 

In [12]:
score_exams(exams,model='nlp1',normv=0,score=0,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,86,83,87
1,10,30,32,88,86,87
2,12,39,24,90,90,85
3,15,38,16,88,87,86
4,13,43,19,91,89,87


Number of entries: 81

SHORT_ANS
Mean Words: 432
Algo score, swaps needed: 996
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1700, 1495, 1355, 1315, 1452, 1516, 1325, 1391, 1372, 1429]
Pseudo-random, avergae swaps needed: 1499.58184

Q1
Mean Words: 2051
Algo score, swaps needed: 1017
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1491, 1515, 1578, 1845, 1595, 1469, 1474, 1630, 1623, 1494]
Pseudo-random, avergae swaps needed: 1566.40308

Q2
Mean Words: 952
Algo score, swaps needed: 828
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1587, 1601, 1617, 1602, 1523, 1586, 1553, 1447, 1638, 1606]
Pseudo-random, avergae swaps needed: 1560.04311

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,88,86
1,20.0,18.0,80,84
2,24.0,25.0,89,80
3,18.0,22.0,83,91
4,17.0,17.0,78,79


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1290
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1923, 1832, 1751, 1906, 1829, 1808, 1952, 1876, 1874, 2022]
Pseudo-random, avergae swaps needed: 1837.64285

Q2
Mean Words: 1546
Algo score, swaps needed: 1326
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1697, 1856, 1533, 1525, 1863, 1795, 1705, 1788, 1810, 2022]
Pseudo-random, avergae swaps needed: 1757.02016

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 911
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1139, 1220, 1159, 1277, 1377, 1528, 1320, 1326, 1365, 1313]
Pseudo-random, avergae swaps needed: 1324.60782

Q2
Mean Words: 570
Algo score, swaps needed: 795
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1303, 1245, 1125, 1247, 1248, 967, 1129, 1268, 1110, 1159]
Pseudo-random, avergae swaps needed: 1219.40096

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,87,87,81
1,53,27,27,82,80,89
2,38,36,20,81,84,81
3,66,37,37,83,88,87
4,71,34,38,85,88,85


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1178
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1362, 1353, 1523, 1521, 1302, 1454, 1544, 1360, 1572, 1280]
Pseudo-random, avergae swaps needed: 1472.10548

Q2
Mean Words: 1050
Algo score, swaps needed: 993
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1423, 1290, 1422, 1431, 1508, 1334, 1202, 1580, 1668, 1328]
Pseudo-random, avergae swaps needed: 1449.81652

Q3
Mean Words: 839
Algo score, swaps needed: 807
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1368, 1352, 1542, 1247, 1578, 1522, 1530, 1636, 1376, 1465]
Pseudo-random, avergae swaps needed: 1458.29175

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,89,89
1,24.0,25.0,82,85
2,26.5,24.0,63,75
3,23.5,23.0,87,87
4,26.5,21.0,87,90


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1792
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1772, 2187, 1756, 1938, 1960, 1940, 2019, 2184, 2151, 2032]
Pseudo-random, avergae swaps needed: 2044.94979

Q2
Mean Words: 2137
Algo score, swaps needed: 1637
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1875, 1873, 1822, 2109, 1681, 1682, 2092, 1945, 2084, 1990]
Pseudo-random, avergae swaps needed: 1994.41962

Swaps (Machine):
[996, 1017, 828, 1290, 1326, 115, 911, 795, 1178, 993, 807, 1792, 1637]
N: 13 	Mean:  1052.6923076923076 	Var:  163027.44378698224
Avergae swaps (Pseudo-random):
[1499.58184, 1566.40308, 1560.04311, 1837.64285, 1757.02016, 182.62802, 1324.60782, 1219.40096, 1472.10548, 1449.81652, 1458.29175, 2044.94979, 1994.41962]
N: 13 	Mean:  1489.7623846153847 	Var:  198318.34859942732 

1 ) 1499.58184 > 996 => 40.35787021114083
2 ) 1566.40308 > 1017 => 42.53328365622294
3 ) 1560.04311 > 828 => 61.30903641852596
4 ) 1837.642

The following cells present a variety of parameters selections, including a mix of vector normalization, grade translations, "best answer" selections, and the number of pseudo-random runs.

In [13]:
score_exams(exams,model='nlp1',normv=1,score=0,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,87,85,86
1,10,30,32,87,83,86
2,12,39,24,89,90,85
3,15,38,16,88,86,86
4,13,43,19,92,89,87


Number of entries: 81

SHORT_ANS
Mean Words: 432
Algo score, swaps needed: 1005
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1473, 1290, 1635, 1584, 1487, 1451, 1823, 1688, 1680, 1445]
Pseudo-random, avergae swaps needed: 1500.17849

Q1
Mean Words: 2051
Algo score, swaps needed: 1056
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1728, 1487, 1621, 1613, 1786, 1575, 1770, 1539, 1534, 1584]
Pseudo-random, avergae swaps needed: 1566.75821

Q2
Mean Words: 952
Algo score, swaps needed: 963
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1785, 1733, 1582, 1374, 1640, 1478, 1617, 1521, 1607, 1461]
Pseudo-random, avergae swaps needed: 1559.79061

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,85,83
1,20.0,18.0,79,83
2,24.0,25.0,89,79
3,18.0,22.0,83,92
4,17.0,17.0,82,79


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1255
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2029, 1947, 2129, 1944, 1753, 1647, 1756, 1569, 1782, 1962]
Pseudo-random, avergae swaps needed: 1837.26637

Q2
Mean Words: 1546
Algo score, swaps needed: 1215
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1689, 1810, 1566, 1850, 1586, 1704, 1639, 1516, 1804, 1846]
Pseudo-random, avergae swaps needed: 1758.08818

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 921
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1403, 1347, 1267, 1091, 1527, 1340, 1486, 1223, 1407, 1317]
Pseudo-random, avergae swaps needed: 1324.18605

Q2
Mean Words: 570
Algo score, swaps needed: 891
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1155, 1247, 1146, 1429, 1220, 1239, 1270, 1185, 1348, 1194]
Pseudo-random, avergae swaps needed: 1219.63849

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,87,85,80
1,53,27,27,81,79,89
2,38,36,20,81,83,81
3,66,37,37,82,88,87
4,71,34,38,85,91,83


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1189
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1493, 1515, 1449, 1279, 1527, 1312, 1335, 1308, 1346, 1503]
Pseudo-random, avergae swaps needed: 1471.99546

Q2
Mean Words: 1050
Algo score, swaps needed: 1049
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1373, 1429, 1272, 1577, 1367, 1446, 1463, 1420, 1468, 1384]
Pseudo-random, avergae swaps needed: 1449.92206

Q3
Mean Words: 839
Algo score, swaps needed: 814
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1463, 1655, 1638, 1379, 1711, 1376, 1354, 1382, 1724, 1440]
Pseudo-random, avergae swaps needed: 1457.47583

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,90,91
1,24.0,25.0,81,84
2,26.5,24.0,61,74
3,23.5,23.0,85,85
4,26.5,21.0,86,91


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1658
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2078, 2060, 1989, 2029, 2156, 1979, 2143, 2160, 1879, 2113]
Pseudo-random, avergae swaps needed: 2045.47186

Q2
Mean Words: 2137
Algo score, swaps needed: 1580
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2002, 1907, 2077, 1890, 2179, 2282, 2104, 2099, 1772, 2292]
Pseudo-random, avergae swaps needed: 1994.37164

Swaps (Machine):
[1005, 1056, 963, 1255, 1215, 113, 921, 891, 1189, 1049, 814, 1658, 1580]
N: 13 	Mean:  1054.5384615384614 	Var:  132621.94082840235
Avergae swaps (Pseudo-random):
[1500.17849, 1566.75821, 1559.79061, 1837.26637, 1758.08818, 182.53896, 1324.18605, 1219.63849, 1471.99546, 1449.92206, 1457.47583, 2045.47186, 1994.37164]
N: 13 	Mean:  1489.8217084615387 	Var:  198407.91481008907 

1 ) 1500.17849 > 1005 => 39.53239196142068
2 ) 1566.75821 > 1056 => 38.94817357182155
3 ) 1559.79061 > 963 => 47.31194159629443
4 ) 1837.

In [14]:
score_exams(exams,model='nlp1',normv=1,score=1,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,87,83,83
1,10,30,32,83,83,83
2,12,39,24,87,87,83
3,15,38,16,87,83,83
4,13,43,19,90,87,83


Number of entries: 81

SHORT_ANS
Mean Words: 432
Algo score, swaps needed: 996
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1522, 1412, 1378, 1620, 1652, 1531, 1796, 1523, 1457, 1647]
Pseudo-random, avergae swaps needed: 1499.41866

Q1
Mean Words: 2051
Algo score, swaps needed: 1060
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1301, 1442, 1555, 1589, 1484, 1718, 1346, 1648, 1346, 1614]
Pseudo-random, avergae swaps needed: 1566.1252

Q2
Mean Words: 952
Algo score, swaps needed: 948
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1472, 1654, 1684, 1658, 1476, 1589, 1584, 1698, 1618, 1519]
Pseudo-random, avergae swaps needed: 1560.14776

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,83,83
1,20.0,18.0,77,80
2,24.0,25.0,87,77
3,18.0,22.0,80,90
4,17.0,17.0,80,77


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1338
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1844, 1911, 1780, 1809, 1924, 1777, 1964, 2069, 1880, 1611]
Pseudo-random, avergae swaps needed: 1837.96154

Q2
Mean Words: 1546
Algo score, swaps needed: 1254
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1625, 1737, 1791, 1544, 1673, 1825, 1833, 2128, 1654, 1713]
Pseudo-random, avergae swaps needed: 1757.82504

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 904
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1365, 1213, 1264, 1480, 1149, 1473, 1407, 1145, 1368, 1494]
Pseudo-random, avergae swaps needed: 1324.80938

Q2
Mean Words: 570
Algo score, swaps needed: 1046
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1229, 1102, 1366, 1390, 1166, 1373, 1153, 1209, 1369, 1340]
Pseudo-random, avergae swaps needed: 1219.48484

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,87,83,80
1,53,27,27,80,77,87
2,38,36,20,80,80,80
3,66,37,37,80,87,83
4,71,34,38,83,90,80


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1174
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1626, 1467, 1423, 1380, 1491, 1176, 1550, 1296, 1383, 1452]
Pseudo-random, avergae swaps needed: 1472.50053

Q2
Mean Words: 1050
Algo score, swaps needed: 1084
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1500, 1583, 1454, 1559, 1604, 1492, 1348, 1427, 1522, 1459]
Pseudo-random, avergae swaps needed: 1449.74993

Q3
Mean Words: 839
Algo score, swaps needed: 784
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1540, 1445, 1417, 1546, 1398, 1260, 1525, 1546, 1177, 1557]
Pseudo-random, avergae swaps needed: 1458.02978

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,90,90
1,24.0,25.0,80,83
2,26.5,24.0,60,73
3,23.5,23.0,83,83
4,26.5,21.0,83,90


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1627
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1969, 2197, 1945, 2015, 2073, 2119, 1960, 2210, 2066, 1894]
Pseudo-random, avergae swaps needed: 2046.44388

Q2
Mean Words: 2137
Algo score, swaps needed: 1622
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1823, 1705, 1973, 1737, 1947, 2071, 2090, 2138, 1763, 1980]
Pseudo-random, avergae swaps needed: 1993.45103

Swaps (Machine):
[996, 1060, 948, 1338, 1254, 102, 904, 1046, 1174, 1084, 784, 1627, 1622]
N: 13 	Mean:  1072.2307692307693 	Var:  138384.02366863904
Avergae swaps (Pseudo-random):
[1499.41866, 1566.1252, 1560.14776, 1837.96154, 1757.82504, 182.46743, 1324.80938, 1219.48484, 1472.50053, 1449.74993, 1458.02978, 2046.44388, 1993.45103]
N: 13 	Mean:  1489.878076923077 	Var:  198443.1305902011 

1 ) 1499.41866 > 996 => 40.3474309196678
2 ) 1566.1252 > 1060 => 38.54539760709048
3 ) 1560.14776 > 948 => 48.81273502004524
4 ) 1837.96154 

In [15]:
score_exams(exams,model='nlp1',normv=1,score=1,goal="medoid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,83,83,83
1,10,30,32,83,77,83
2,12,39,24,83,87,83
3,15,38,16,87,87,83
4,13,43,19,97,90,83


Number of entries: 81

SHORT_ANS
Mean Words: 432
Algo score, swaps needed: 1131
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1508, 1669, 1589, 1620, 1649, 1595, 1501, 1371, 1259, 1565]
Pseudo-random, avergae swaps needed: 1498.76345

Q1
Mean Words: 2051
Algo score, swaps needed: 1092
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1688, 1679, 1599, 1705, 1546, 1655, 1700, 1690, 1650, 1630]
Pseudo-random, avergae swaps needed: 1566.83956

Q2
Mean Words: 952
Algo score, swaps needed: 992
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1646, 1363, 1668, 1550, 1597, 1406, 1471, 1586, 1687, 1539]
Pseudo-random, avergae swaps needed: 1559.60799

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,83,83
1,20.0,18.0,80,77
2,24.0,25.0,87,80
3,18.0,22.0,80,90
4,17.0,17.0,80,87


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1292
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1719, 1727, 2007, 1807, 1956, 1626, 1871, 1721, 1622, 1671]
Pseudo-random, avergae swaps needed: 1837.8108

Q2
Mean Words: 1546
Algo score, swaps needed: 1607
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1574, 1563, 1420, 1712, 1739, 2043, 1736, 1662, 1659, 1936]
Pseudo-random, avergae swaps needed: 1757.76423

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 885
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1211, 1365, 1332, 1370, 1252, 1348, 1445, 1335, 1292, 1382]
Pseudo-random, avergae swaps needed: 1324.15575

Q2
Mean Words: 570
Algo score, swaps needed: 1036
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1440, 1239, 1101, 1082, 1098, 1361, 1247, 1181, 1303, 1162]
Pseudo-random, avergae swaps needed: 1218.86026

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,87,80,77
1,53,27,27,77,80,87
2,38,36,20,77,83,80
3,66,37,37,80,87,83
4,71,34,38,83,90,83


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1283
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1625, 1383, 1504, 1201, 1228, 1432, 1426, 1473, 1476, 1282]
Pseudo-random, avergae swaps needed: 1473.32161

Q2
Mean Words: 1050
Algo score, swaps needed: 1158
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1434, 1475, 1506, 1553, 1561, 1324, 1406, 1611, 1232, 1085]
Pseudo-random, avergae swaps needed: 1450.12534

Q3
Mean Words: 839
Algo score, swaps needed: 927
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1358, 1516, 1299, 1382, 1450, 1380, 1278, 1514, 1564, 1545]
Pseudo-random, avergae swaps needed: 1458.40317

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,83,90
1,24.0,25.0,77,80
2,26.5,24.0,63,70
3,23.5,23.0,80,80
4,26.5,21.0,83,90


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1846
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2061, 1860, 2304, 1984, 2021, 1818, 2021, 2240, 1989, 1924]
Pseudo-random, avergae swaps needed: 2045.38608

Q2
Mean Words: 2137
Algo score, swaps needed: 1645
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1945, 2114, 1689, 2050, 2059, 1552, 1779, 1790, 1823, 2221]
Pseudo-random, avergae swaps needed: 1993.78851

Swaps (Machine):
[1131, 1092, 992, 1292, 1607, 140, 885, 1036, 1283, 1158, 927, 1846, 1645]
N: 13 	Mean:  1156.4615384615386 	Var:  165954.0946745562
Avergae swaps (Pseudo-random):
[1498.76345, 1566.83956, 1559.60799, 1837.8108, 1757.76423, 182.45657, 1324.15575, 1218.86026, 1473.32161, 1450.12534, 1458.40317, 2045.38608, 1993.78851]
N: 13 	Mean:  1489.7910246153845 	Var:  198408.4483684661 

1 ) 1498.76345 > 1131 => 27.9693179247738
2 ) 1566.83956 > 1092 => 35.71780465008576
3 ) 1559.60799 > 992 => 44.490218891343105
4 ) 1837.81

In [16]:
score_exams(exams,model='nlp2',normv=1,score=1,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,87,83,87
1,10,30,32,87,83,87
2,12,39,24,90,80,83
3,15,38,16,90,83,87
4,13,43,19,90,83,83


Number of entries: 81

SHORT_ANS
Mean Words: 432
Algo score, swaps needed: 1003
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1442, 1394, 1469, 1617, 1478, 1532, 1515, 1422, 1649, 1390]
Pseudo-random, avergae swaps needed: 1499.03329

Q1
Mean Words: 2051
Algo score, swaps needed: 1199
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1686, 1439, 1483, 1586, 1523, 1611, 1525, 1516, 1490, 1552]
Pseudo-random, avergae swaps needed: 1566.15807

Q2
Mean Words: 952
Algo score, swaps needed: 907
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1533, 1567, 1755, 1751, 1638, 1542, 1484, 1657, 1444, 1578]
Pseudo-random, avergae swaps needed: 1560.30731

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,83,87
1,20.0,18.0,70,80
2,24.0,25.0,90,80
3,18.0,22.0,77,87
4,17.0,17.0,77,80


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1293
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1786, 2004, 1738, 2214, 1852, 1685, 1676, 1809, 1739, 1956]
Pseudo-random, avergae swaps needed: 1837.47318

Q2
Mean Words: 1546
Algo score, swaps needed: 1274
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1540, 1645, 1793, 1887, 1574, 1840, 1970, 1518, 1773, 1717]
Pseudo-random, avergae swaps needed: 1757.25757

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 932
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1202, 1263, 1181, 1474, 1288, 1465, 1433, 1470, 1378, 1313]
Pseudo-random, avergae swaps needed: 1324.22136

Q2
Mean Words: 570
Algo score, swaps needed: 839
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1240, 1212, 1178, 1266, 1296, 1194, 1299, 1237, 1324, 1171]
Pseudo-random, avergae swaps needed: 1219.99053

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,77,80,80
1,53,27,27,83,87,87
2,38,36,20,77,80,80
3,66,37,37,83,83,83
4,71,34,38,83,87,87


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1125
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1636, 1480, 1496, 1540, 1297, 1366, 1305, 1360, 1362, 1567]
Pseudo-random, avergae swaps needed: 1472.33798

Q2
Mean Words: 1050
Algo score, swaps needed: 944
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1268, 1592, 1590, 1324, 1410, 1433, 1482, 1538, 1341, 1471]
Pseudo-random, avergae swaps needed: 1449.79584

Q3
Mean Words: 839
Algo score, swaps needed: 843
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1438, 1577, 1425, 1461, 1424, 1423, 1327, 1611, 1645, 1471]
Pseudo-random, avergae swaps needed: 1458.01243

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,93,90
1,24.0,25.0,87,87
2,26.5,24.0,77,80
3,23.5,23.0,90,87
4,26.5,21.0,83,83


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1751
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2066, 1880, 2144, 2034, 1986, 2242, 1992, 2192, 1907, 2143]
Pseudo-random, avergae swaps needed: 2045.28171

Q2
Mean Words: 2137
Algo score, swaps needed: 1716
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1835, 1950, 2200, 2172, 1806, 2043, 2065, 1929, 1891, 1921]
Pseudo-random, avergae swaps needed: 1993.56841

Swaps (Machine):
[1003, 1199, 907, 1293, 1274, 87, 932, 839, 1125, 944, 843, 1751, 1716]
N: 13 	Mean:  1070.2307692307693 	Var:  163788.02366863904
Avergae swaps (Pseudo-random):
[1499.03329, 1566.15807, 1560.30731, 1837.47318, 1757.25757, 182.42954, 1324.22136, 1219.99053, 1472.33798, 1449.79584, 1458.01243, 2045.28171, 1993.56841]
N: 13 	Mean:  1489.6820938461537 	Var:  198306.63860080714 

1 ) 1499.03329 > 1003 => 39.65041488316888
2 ) 1566.15807 > 1199 => 26.556027590856672
3 ) 1560.30731 > 907 => 52.95710893832677
4 ) 1837.4

In [17]:
score_exams(exams,model='nlp2',normv=1,score=1,goal="medoid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,83,80,83
1,10,30,32,83,80,90
2,12,39,24,90,83,83
3,15,38,16,87,83,83
4,13,43,19,87,83,80


Number of entries: 81

SHORT_ANS
Mean Words: 432
Algo score, swaps needed: 1032
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1565, 1445, 1770, 1410, 1545, 1578, 1481, 1447, 1520, 1423]
Pseudo-random, avergae swaps needed: 1499.06997

Q1
Mean Words: 2051
Algo score, swaps needed: 1217
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1604, 1587, 1647, 1574, 1518, 1388, 1565, 1577, 1713, 1626]
Pseudo-random, avergae swaps needed: 1566.32602

Q2
Mean Words: 952
Algo score, swaps needed: 867
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1498, 1736, 1475, 1473, 1534, 1700, 1755, 1792, 1572, 1596]
Pseudo-random, avergae swaps needed: 1559.19013

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,83,87
1,20.0,18.0,73,80
2,24.0,25.0,90,80
3,18.0,22.0,77,87
4,17.0,17.0,77,80


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1230
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2073, 1812, 1731, 2097, 1634, 1963, 1935, 1819, 1657, 1916]
Pseudo-random, avergae swaps needed: 1837.24763

Q2
Mean Words: 1546
Algo score, swaps needed: 1202
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1627, 1896, 1774, 1877, 2033, 1685, 1742, 2005, 1713, 1656]
Pseudo-random, avergae swaps needed: 1758.28921

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 1016
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1362, 1552, 1157, 1378, 1269, 1326, 1371, 1442, 1520, 1189]
Pseudo-random, avergae swaps needed: 1324.05922

Q2
Mean Words: 570
Algo score, swaps needed: 1001
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1289, 1339, 1220, 1154, 1049, 1049, 1192, 1201, 1342, 1316]
Pseudo-random, avergae swaps needed: 1219.84503

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,83,83,80
1,53,27,27,80,83,87
2,38,36,20,80,77,77
3,66,37,37,83,80,83
4,71,34,38,83,87,87


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1066
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1453, 1281, 1283, 1448, 1376, 1415, 1449, 1582, 1451, 1439]
Pseudo-random, avergae swaps needed: 1472.55302

Q2
Mean Words: 1050
Algo score, swaps needed: 1017
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1537, 1417, 1434, 1618, 1354, 1462, 1385, 1538, 1402, 1644]
Pseudo-random, avergae swaps needed: 1449.81932

Q3
Mean Words: 839
Algo score, swaps needed: 758
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1363, 1525, 1691, 1160, 1467, 1161, 1453, 1530, 1855, 1497]
Pseudo-random, avergae swaps needed: 1457.66226

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,90,90
1,24.0,25.0,90,87
2,26.5,24.0,80,80
3,23.5,23.0,87,90
4,26.5,21.0,87,83


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1769
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2125, 2102, 2110, 2147, 2384, 2151, 2236, 2088, 2282, 2069]
Pseudo-random, avergae swaps needed: 2044.82171

Q2
Mean Words: 2137
Algo score, swaps needed: 1696
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1723, 2100, 1988, 2046, 1962, 2028, 2042, 1689, 1833, 1822]
Pseudo-random, avergae swaps needed: 1994.01532

Swaps (Machine):
[1032, 1217, 867, 1230, 1202, 82, 1016, 1001, 1066, 1017, 758, 1769, 1696]
N: 13 	Mean:  1073.3076923076924 	Var:  159354.67455621302
Avergae swaps (Pseudo-random):
[1499.06997, 1566.32602, 1559.19013, 1837.24763, 1758.28921, 182.47197, 1324.05922, 1219.84503, 1472.55302, 1449.81932, 1457.66226, 2044.82171, 1994.01532]
N: 13 	Mean:  1489.643908461538 	Var:  198325.10414515517 

1 ) 1499.06997 > 1032 => 36.906918855348756
2 ) 1566.32602 > 1217 => 25.10133685309348
3 ) 1559.19013 > 867 => 57.05984221442694
4 ) 1837

In [18]:
score_exams(exams,model='nlp3',normv=1,score=1,goal="centroid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,83,87,83
1,10,30,32,87,83,87
2,12,39,24,87,67,80
3,15,38,16,87,87,87
4,13,43,19,80,83,83


Number of entries: 81

SHORT_ANS
Mean Words: 432
Algo score, swaps needed: 1300
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1496, 1343, 1545, 1325, 1615, 1482, 1539, 1426, 1587, 1560]
Pseudo-random, avergae swaps needed: 1499.78593

Q1
Mean Words: 2051
Algo score, swaps needed: 1250
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1542, 1606, 1524, 1587, 1699, 1412, 1598, 1573, 1449, 1633]
Pseudo-random, avergae swaps needed: 1566.66666

Q2
Mean Words: 952
Algo score, swaps needed: 1170
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1455, 1778, 1608, 1510, 1596, 1547, 1467, 1823, 1317, 1662]
Pseudo-random, avergae swaps needed: 1559.28025

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,87,87
1,20.0,18.0,77,83
2,24.0,25.0,90,87
3,18.0,22.0,83,77
4,17.0,17.0,77,87


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1606
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1967, 1918, 1774, 1843, 1888, 1841, 2038, 1691, 1568, 1700]
Pseudo-random, avergae swaps needed: 1837.66504

Q2
Mean Words: 1546
Algo score, swaps needed: 1570
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1856, 1673, 1855, 1739, 1900, 1994, 1622, 1871, 1793, 1911]
Pseudo-random, avergae swaps needed: 1757.84384

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 927
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1172, 1283, 1480, 1384, 1241, 1281, 1487, 1495, 1319, 1293]
Pseudo-random, avergae swaps needed: 1324.76258

Q2
Mean Words: 570
Algo score, swaps needed: 1013
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1212, 1515, 986, 1241, 1137, 1207, 1202, 1253, 1119, 1097]
Pseudo-random, avergae swaps needed: 1219.60677

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,80,83,83
1,53,27,27,83,87,83
2,38,36,20,87,83,83
3,66,37,37,83,83,83
4,71,34,38,83,87,80


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1402
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1540, 1566, 1248, 1558, 1729, 1528, 1481, 1579, 1509, 1589]
Pseudo-random, avergae swaps needed: 1472.71575

Q2
Mean Words: 1050
Algo score, swaps needed: 1187
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1380, 1481, 1375, 1418, 1514, 1429, 1413, 1460, 1281, 1426]
Pseudo-random, avergae swaps needed: 1449.86842

Q3
Mean Words: 839
Algo score, swaps needed: 1073
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1363, 1423, 1429, 1343, 1417, 1609, 1485, 1489, 1660, 1446]
Pseudo-random, avergae swaps needed: 1457.79744

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,90,90
1,24.0,25.0,87,83
2,26.5,24.0,77,83
3,23.5,23.0,87,87
4,26.5,21.0,87,87


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1668
Swaps needed for first 10 out of 100000 pseudo-random runs:
[2165, 2055, 2052, 2092, 1928, 2020, 1906, 1937, 2313, 2215]
Pseudo-random, avergae swaps needed: 2045.93318

Q2
Mean Words: 2137
Algo score, swaps needed: 1746
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1881, 1744, 1849, 2311, 1922, 1760, 2161, 1922, 2347, 1731]
Pseudo-random, avergae swaps needed: 1994.50546

Swaps (Machine):
[1300, 1250, 1170, 1606, 1570, 114, 927, 1013, 1402, 1187, 1073, 1668, 1746]
N: 13 	Mean:  1232.7692307692307 	Var:  166485.5621301775
Avergae swaps (Pseudo-random):
[1499.78593, 1566.66666, 1559.28025, 1837.66504, 1757.84384, 182.38322, 1324.76258, 1219.60677, 1472.71575, 1449.86842, 1457.79744, 2045.93318, 1994.50546]
N: 13 	Mean:  1489.9088107692307 	Var:  198476.59919975683 

1 ) 1499.78593 > 1300 => 14.27151467969553
2 ) 1566.66666 > 1250 => 22.485206680438367
3 ) 1559.28025 > 1170 => 28.52622042020053
4 ) 1

In [19]:
score_exams(exams,model='nlp3',normv=1,score=1,goal="medoid",runs=100000)


../data/property_instructor_A

['ID', 'SHORT_ANS', 'Q1', 'Q2']


,SHORT_ANS_x,Q1_x,Q2_x,SHORT_ANS_y,Q1_y,Q2_y
0,15,44,24,80,87,83
1,10,30,32,87,87,87
2,12,39,24,87,67,77
3,15,38,16,87,87,83
4,13,43,19,83,83,80


Number of entries: 81

SHORT_ANS
Mean Words: 432
Algo score, swaps needed: 1259
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1592, 1489, 1815, 1469, 1572, 1183, 1538, 1453, 1557, 1711]
Pseudo-random, avergae swaps needed: 1499.24829

Q1
Mean Words: 2051
Algo score, swaps needed: 1183
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1640, 1351, 1636, 1642, 1648, 1636, 1610, 1648, 1241, 1402]
Pseudo-random, avergae swaps needed: 1566.52114

Q2
Mean Words: 952
Algo score, swaps needed: 1220
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1452, 1388, 1783, 1489, 1631, 1360, 1697, 1651, 1776, 1683]
Pseudo-random, avergae swaps needed: 1560.17839

../data/property_instructor_B

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,19.0,20.0,90,87
1,20.0,18.0,80,77
2,24.0,25.0,90,83
3,18.0,22.0,87,73
4,17.0,17.0,77,87


Number of entries: 88

Q1
Mean Words: 1431
Algo score, swaps needed: 1558
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1736, 1850, 1896, 1597, 1999, 1797, 1912, 1929, 1861, 2036]
Pseudo-random, avergae swaps needed: 1837.75455

Q2
Mean Words: 1546
Algo score, swaps needed: 1489
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1719, 1509, 1635, 1800, 1828, 1835, 1972, 1966, 2069, 1983]
Pseudo-random, avergae swaps needed: 1757.5462

../data/environ_instructor_B

['ID', 'Q2']


...

Number of entries: 75

Q1
Mean Words: 936
Algo score, swaps needed: 824
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1338, 1458, 1417, 1202, 1405, 1149, 1314, 1416, 1273, 1173]
Pseudo-random, avergae swaps needed: 1324.74177

Q2
Mean Words: 570
Algo score, swaps needed: 920
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1224, 1348, 1333, 1173, 1369, 1379, 1335, 1176, 1182, 1141]
Pseudo-random, avergae swaps needed: 1219.22249

../data/contracts_instructor_D

['ID', 'Q1', 'Q2', 'Q3']


,Q1_x,Q2_x,Q3_x,Q1_y,Q2_y,Q3_y
0,43,25,16,80,83,87
1,53,27,27,83,87,83
2,38,36,20,87,83,83
3,66,37,37,83,87,83
4,71,34,38,87,90,80


Number of entries: 78

Q1
Mean Words: 1823
Algo score, swaps needed: 1325
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1354, 1377, 1470, 1322, 1628, 1432, 1458, 1437, 1377, 1510]
Pseudo-random, avergae swaps needed: 1471.70375

Q2
Mean Words: 1050
Algo score, swaps needed: 1179
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1499, 1512, 1630, 1522, 1419, 1404, 1284, 1510, 1425, 1257]
Pseudo-random, avergae swaps needed: 1449.50764

Q3
Mean Words: 839
Algo score, swaps needed: 1102
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1471, 1495, 1457, 1477, 1687, 1334, 1414, 1393, 1467, 1456]
Pseudo-random, avergae swaps needed: 1458.83258

../data/crim_instructor_E

['ID', 'Q1', 'Q2']


,Q1_x,Q2_x,Q1_y,Q2_y
0,24.5,24.0,93,90
1,24.0,25.0,90,83
2,26.5,24.0,73,83
3,23.5,23.0,87,87
4,26.5,21.0,90,87


Number of entries: 92

Q1
Mean Words: 3353
Algo score, swaps needed: 1768
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1937, 2116, 2237, 2171, 1637, 2182, 2021, 2128, 1840, 1862]
Pseudo-random, avergae swaps needed: 2045.62298

Q2
Mean Words: 2137
Algo score, swaps needed: 1714
Swaps needed for first 10 out of 100000 pseudo-random runs:
[1777, 2078, 2061, 2132, 2067, 1781, 1872, 1853, 2001, 2057]
Pseudo-random, avergae swaps needed: 1993.49654

Swaps (Machine):
[1259, 1183, 1220, 1558, 1489, 81, 824, 920, 1325, 1179, 1102, 1768, 1714]
N: 13 	Mean:  1201.6923076923076 	Var:  176941.90532544377
Avergae swaps (Pseudo-random):
[1499.24829, 1566.52114, 1560.17839, 1837.75455, 1757.5462, 182.46232, 1324.74177, 1219.22249, 1471.70375, 1449.50764, 1458.83258, 2045.62298, 1993.49654]
N: 13 	Mean:  1489.7568184615386 	Var:  198372.03310967892 

1 ) 1499.24829 > 1259 => 17.42035268333294
2 ) 1566.52114 > 1183 => 27.897304328418446
3 ) 1560.17839 > 1220 => 24.47169514183585
4 ) 183

[back to contents](#Contents)